In [1]:
import os
#cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import __version__ as mpv
import seaborn as sns
from sklearn.base import BaseEstimator,TransformerMixin
import scipy.sparse as sparse
from sklearn.model_selection import train_test_split

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from collections import defaultdict
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
## https://surprise.readthedocs.io/en/stable/FAQ.html
## https://towardsdatascience.com/how-you-can-build-simple-recommender-systems-with-surprise-b0d32a8e4802

In [2]:
rating=pd.read_csv("dataset/ratings.csv")
link=pd.read_csv("dataset/links.csv")
movies=pd.read_csv("dataset/movies.csv")
#tags=pd.read_csv("dataset/tags.csv")
#genome_scores=pd.read_csv("dataset/genome-scores.csv")
#genome_tags=pd.read_csv("dataset/genome-tags.csv")
metadata = pd.read_csv("dataset/movies_metadata.csv")

rating = rating[:100000]

#movielens_dict={v:k for k,v in link.movieId.to_dict().items()}
#rating.movieId=rating.movieId.apply(movielens_dict.get)
#rating.userId-=1
#genome_scores.movieId=genome_scores.movieId.apply(movielens_dict.get)
#genome_scores.tagId-=1
#genome_tags.tagId-=1

/home/sanjay/anaconda3/envs/bigdata/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# svd = SVD(verbose=True)
# svd.predict(uid=10, iid=100)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rating[['userId', 'movieId', 'rating']], reader)

In [4]:
svd = SVD(verbose=True, n_epochs=2)
#cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [5]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1


In [6]:
svd.predict(uid=10, iid=100)

Prediction(uid=10, iid=100, r_ui=None, est=3.3711255864666665, details={'was_impossible': False})

In [7]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [8]:
# algo = SVD()
# algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
#testset = trainset.build_anti_testset()
testset = trainset.build_testset()
print("Completed Testset generation")
predictions = svd.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Completed Testset generation
1 [296, 4973, 5952, 7361, 3949, 6016, 306, 6377, 7940, 899]
2 [318, 527, 1197, 260, 1198, 457, 1196, 4993, 2762, 1136]
3 [50, 318, 858, 2858, 296, 527, 1198, 2571, 593, 260]
4 [260, 1196, 58559, 1198, 2571, 296, 589, 1197, 74458, 79132]
5 [356, 593, 50, 296, 260, 47, 1197, 457, 1210, 1196]
6 [318, 593, 260, 2858, 858, 608, 1196, 527, 912, 1704]
7 [527, 296, 593, 457, 150, 588, 590, 265, 17, 595]
8 [50, 593, 318, 1213, 1196, 110, 1198, 608, 260, 356]
9 [858, 1198, 260, 1221, 541, 589, 1210, 1196, 1197, 593]
10 [260, 318, 593, 1196, 50, 296, 110, 457, 1210, 356]
11 [527, 593, 7153, 4993, 1207, 2858, 2329, 5618, 1203, 2581]
12 [318, 50, 527, 858, 260, 47, 2959, 593, 4973, 2858]
13 [2571, 4226, 296, 4993, 1198, 1270, 318, 260, 1214, 2959]
14 [50, 296, 608, 2959, 2858, 527, 293, 4011, 2329, 4993]
15 [296, 2858, 356, 2571, 1198, 1197, 110, 608, 1968, 2997]
16 [318, 858, 1221, 1193, 2858, 2959, 1704, 4993, 2324, 31658]
17 [134130, 111759, 588, 79091, 47099, 6377, 

In [9]:
metadata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [10]:
pd.set_option('display.max_colwidth', None)
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


### Add Year column from release date

In [11]:
metadata['year'] = pd.to_datetime(metadata['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)


In [12]:
def to_int(x):
    try:
        return int(x)
    except:
        return np.nan

def get_first_index(idx):
    if isinstance(idx, list) or isinstance(idx, pd.Series):
        idx = idx[0]
    return idx        

### Convert Id column to int from object to connect with links

In [13]:
metadata['id'] = metadata['id'].apply(to_int)

In [14]:
def get_top_weighted_rating(df, number_of_records=200, percentile=0.85):
    non_null_vote_counts = df[df['vote_count'].notnull()]['vote_count']
    non_null_vote_avgs = df[df['vote_average'].notnull()]['vote_average']
    mean_votes = non_null_vote_avgs.mean()
    min_votes_req = non_null_vote_counts.quantile(percentile)

    selected = df[(df['vote_count'] >= min_votes_req) & (
        df['vote_count'].notnull()) & (df['vote_average'].notnull())]
    selected = selected[['title', 'year',
                         'vote_count', 'vote_average', 'popularity']]
    selected['weighted_rating'] = selected.apply(lambda x: (
        x.vote_count / (x.vote_count + min_votes_req) * x['vote_average']) + (min_votes_req/(min_votes_req + x.vote_count) * mean_votes), axis=1)

    selected = selected.sort_values(
        'weighted_rating', ascending=False).head(250)
    return selected


def build_top_movie_chart(dataframe, genre=None, percentile=0.85):
    if genre is None:
        df = dataframe
    else:
        metadata_temp = dataframe.copy()
        metadata_temp['genres'] = metadata_temp['genres'].fillna('[]').apply(literal_eval).apply(
            lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
        df = metadata_temp.apply(lambda x: pd.Series(
            x['genres']), axis=1).stack().reset_index(level=1, drop=True)
        df.name = 'genre'
        df = metadata_temp.drop('genres', axis=1).join(df)
        df = df[df['genre'] == genre]

    selected = get_top_weighted_rating(df, 200, percentile)

    return selected


### Top 10 movies by wighted rating

In [15]:

build_top_movie_chart(metadata, percentile=0.95).head(10)

,title,year,vote_count,vote_average,popularity,weighted_rating
314,The Shawshank Redemption,1994,8358.0,8.5,51.645403,8.357746
834,The Godfather,1972,6024.0,8.5,41.109264,8.306334
12481,The Dark Knight,2008,12269.0,8.3,123.167259,8.208376
2843,Fight Club,1999,9678.0,8.3,63.869599,8.184899
292,Pulp Fiction,1994,8670.0,8.3,140.950236,8.172155
351,Forrest Gump,1994,8147.0,8.2,48.307194,8.069421
522,Schindler's List,1993,4436.0,8.3,41.725123,8.061007
23673,Whiplash,2014,4376.0,8.3,64.29999,8.058025
5481,Spirited Away,2001,3968.0,8.3,41.048867,8.035598
1154,The Empire Strikes Back,1980,5998.0,8.2,19.470959,8.025793


### Top 10 Crime Movies

In [16]:
build_top_movie_chart(metadata, genre="Crime", percentile=0.90).head(10)

/home/sanjay/anaconda3/envs/bigdata/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,title,year,vote_count,vote_average,popularity,weighted_rating
314,The Shawshank Redemption,1994,8358.0,8.5,51.645403,8.389613
834,The Godfather,1972,6024.0,8.5,41.109264,8.349302
12481,The Dark Knight,2008,12269.0,8.3,123.167259,8.229593
292,Pulp Fiction,1994,8670.0,8.3,140.950236,8.201554
1178,The Godfather: Part II,1974,3418.0,8.3,36.629307,8.064967
289,Leon: The Professional,1994,4293.0,8.2,20.477329,8.016979
3030,The Green Mile,1999,4166.0,8.2,19.96678,8.011851
46,Se7en,1995,5915.0,8.1,18.45743,7.970079
1170,GoodFellas,1990,3211.0,8.2,15.424092,7.961638
586,The Silence of the Lambs,1991,4549.0,8.1,4.307222,7.933982


### Top 10 Drama movies

In [17]:
build_top_movie_chart(metadata, genre="Drama", percentile=0.90).head(10)

/home/sanjay/anaconda3/envs/bigdata/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,title,year,vote_count,vote_average,popularity,weighted_rating
10309,Dilwale Dulhania Le Jayenge,1995,661.0,9.1,34.457024,8.493149
314,The Shawshank Redemption,1994,8358.0,8.5,51.645403,8.452756
834,The Godfather,1972,6024.0,8.5,41.109264,8.434910
12481,The Dark Knight,2008,12269.0,8.3,123.167259,8.270123
2843,Fight Club,1999,9678.0,8.3,63.869599,8.262251
522,Schindler's List,1993,4436.0,8.3,41.725123,8.219148
23673,Whiplash,2014,4376.0,8.3,64.29999,8.218078
2211,Life Is Beautiful,1997,3643.0,8.3,39.39497,8.202267
1178,The Godfather: Part II,1974,3418.0,8.3,36.629307,8.196112
1152,One Flew Over the Cuckoo's Nest,1975,3001.0,8.3,35.529554,8.182386


In [18]:
metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995


In [19]:
metadata_for_cont = metadata.copy()
metadata_for_cont["tagline"] = metadata_for_cont.tagline.fillna("")
metadata_for_cont["desc"] = metadata_for_cont.overview + \
    metadata_for_cont.tagline
metadata_for_cont["desc"] = metadata_for_cont.desc.fillna("")

metadata_for_cont = metadata_for_cont[metadata_for_cont.id.isin(link.tmdbId)]

In [20]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2),
                     min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(metadata_for_cont.desc)


In [21]:
tfidf_matrix.shape

(42848, 1049589)

In [22]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [23]:
metadata_for_cont = metadata_for_cont.reset_index()
titles = metadata_for_cont['title']
indices = pd.Series(metadata_for_cont.index, index=metadata_for_cont['title'])


In [24]:
def get_recommendations(title):
    idx = get_first_index(indices[title])
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [25]:
get_recommendations('The Family').head(10)

23529                       My Sister and I
15706                         Seven Thieves
29115                       Black and White
26112    Who Killed the Prosecutor and Why?
5853                       Abraham's Valley
26752                         Kiss Me Again
15815                           Monte Carlo
9230                   Remember Me, My Love
32200              Truffaldino from Bergamo
38988                     Fisherman's Wharf
Name: title, dtype: object

In [26]:
get_recommendations('The Apartment').head(10)

17658               Miss Nobody
8943                   LadyBugs
14094                  Birdsong
32824        Unlocking the Cage
29887           Company Limited
7443     Shadow of the Thin Man
8697          The Holy Mountain
2298          Nothing in Common
8352         The Shopworn Angel
21390            Black Nativity
Name: title, dtype: object

In [27]:
def cosine_sim_plus_weighted_rating(title, quantile=0.60):
    idx = get_first_index(indices[title])
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movs = metadata_for_cont.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'popularity']]
    selected = get_top_weighted_rating(movs, 10, 0.60)
    return selected

In [28]:
cosine_sim_plus_weighted_rating('The Family')

,title,year,vote_count,vote_average,popularity,weighted_rating
5358,The Last Kiss,2001,84.0,6.5,5.50323,6.385106
42714,The Olive Tree,2016,38.0,6.5,2.541633,6.275000
20489,Those Daring Young Men in Their Jaunty Jalopies,1969,10.0,6.8,4.336514,6.110000
34492,Violent Summer,1959,10.0,6.7,0.906731,6.060000
16581,Monte Carlo,2011,611.0,6.0,8.34835,5.990660
9230,"Remember Me, My Love",2003,47.0,5.8,4.996071,5.733333
5839,Pinocchio,2002,144.0,5.6,6.463613,5.588312
27721,Stuff for the Rich,1987,17.0,5.5,0.725035,5.470370
29115,Black and White,2008,10.0,5.5,0.943728,5.460000
7020,Once Upon a Crime,1992,17.0,5.3,1.984181,5.344444


In [29]:
cosine_sim_plus_weighted_rating('The Apartment')

,title,year,vote_count,vote_average,popularity,weighted_rating
8697,The Holy Mountain,1973,143.0,7.7,4.190266,7.461539
7642,Safety Last!,1923,76.0,7.7,4.394823,7.302406
1203,The Day the Earth Stood Still,1951,323.0,7.3,9.360003,7.213188
80,Things to Do in Denver When You're Dead,1995,87.0,6.7,4.486179,6.560697
4908,48 Hrs.,1982,364.0,6.5,15.297121,6.472610
2337,Running Scared,1986,58.0,6.4,4.819467,6.300505
3888,The House of Mirth,2000,25.0,6.4,2.492547,6.234040
2298,Nothing in Common,1986,30.0,6.1,8.750517,6.083853
8943,LadyBugs,1992,35.0,5.3,2.386817,5.613832
23663,Leprechaun: Back 2 tha Hood,2003,26.0,4.9,2.551286,5.463524


In [30]:
movieId_tmdbId = link.copy()
movieId_tmdbId = movieId_tmdbId[["movieId", "tmdbId"]]
movieId_tmdbId.columns = ['movieId', 'id']
movieId_tmdbId = movieId_tmdbId.merge(metadata_for_cont[['title', 'id']], on='id').set_index('title')
movieId_tmdbId

,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0
...,...,...
Betrayal,176273,67758.0
Satan Triumphant,176275,227506.0
Queerama,176279,461257.0


In [31]:
tmdbId_index_movie = movieId_tmdbId.set_index('id')

In [32]:
def hybrid_model_cosine_weighted_rate_svd(userId, title):
    idx = get_first_index(indices[title])
    tmdbId = movieId_tmdbId.loc[title]['id']
    #print(idx)
    movie_id = movieId_tmdbId.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = metadata_for_cont.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, tmdbId_index_movie.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [33]:
hybrid_model_cosine_weighted_rate_svd(1, 'The Family')

,title,vote_count,vote_average,year,id,est
16581,Monte Carlo,611.0,6.0,2011,59860.0,3.833655
23529,My Sister and I,2.0,8.0,1987,62491.0,3.677669
15706,Seven Thieves,9.0,6.4,1960,64605.0,3.677669
20489,Those Daring Young Men in Their Jaunty Jalopies,10.0,6.8,1969,38792.0,3.677669
24689,The Law,4.0,5.8,1990,65482.0,3.677669
42714,The Olive Tree,38.0,6.5,2016,366505.0,3.677669
27721,Stuff for the Rich,17.0,5.5,1987,58613.0,3.677669
36574,Brothers and Sisters of the Toda Family,4.0,7.3,1941,94659.0,3.677669
31102,Kill and Pray,3.0,6.7,1967,122787.0,3.677669
34836,Hold That Baby!,0.0,0.0,1949,186775.0,3.677669


In [34]:
hybrid_model_cosine_weighted_rate_svd(2, 'The Apartment')

,title,vote_count,vote_average,year,id,est
7642,Safety Last!,76.0,7.7,1923,22596.0,3.808084
38706,Alesha Popovich and Tugarin the Dragon,20.0,6.1,2004,33065.0,3.743474
7443,Shadow of the Thin Man,17.0,6.2,1941,14590.0,3.726306
1917,The Barefoot Executive,10.0,5.2,1971,20173.0,3.670322
3888,The House of Mirth,25.0,6.4,2000,25520.0,3.634741
4908,48 Hrs.,364.0,6.5,1982,150.0,3.622634
2337,Running Scared,58.0,6.4,1986,15698.0,3.599609
2298,Nothing in Common,30.0,6.1,1986,29968.0,3.587893
30389,The Lorax,16.0,7.8,1972,38060.0,3.586121
17658,Miss Nobody,15.0,4.3,2010,51875.0,3.573468
